# Binary Brains Team

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np


In [2]:
df = pd.read_csv('Datasets/Churn_Modelling.csv')
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [3]:
df.drop(['RowNumber','CustomerId','Surname'], axis= 'columns', inplace=True)
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df.dtypes

CreditScore          int64
Geography           object
Gender              object
Age                  int64
Tenure               int64
Balance            float64
NumOfProducts        int64
HasCrCard            int64
IsActiveMember       int64
EstimatedSalary    float64
Exited               int64
dtype: object

In [5]:
def print_unique_col_values(df):
    for column in df:
        if df[column].dtypes == 'object':
            print(f'{column}: {df[column].unique()}')

In [6]:
print_unique_col_values(df)

Geography: ['France' 'Spain' 'Germany']
Gender: ['Female' 'Male']


In [7]:
df1 = pd.get_dummies(data=df, columns=['Geography', 'Gender'])
df1.columns

Index(['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited', 'Geography_France',
       'Geography_Germany', 'Geography_Spain', 'Gender_Female', 'Gender_Male'],
      dtype='object')

In [8]:
print_unique_col_values(df1)

In [9]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [10]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [11]:
col_to_scale=['CreditScore', 'Age', 'Tenure','Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
df1[col_to_scale] = scaler.fit_transform(df1[col_to_scale])

In [13]:
df1.head()

,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,0.538,0.324324,0.2,0.000000,0.000000,1,1,0.506735,1,1,0,0,1,0
1,0.516,0.310811,0.1,0.334031,0.000000,0,1,0.562709,0,0,0,1,1,0
2,0.304,0.324324,0.8,0.636357,0.666667,1,0,0.569654,1,1,0,0,1,0
3,0.698,0.283784,0.1,0.000000,0.333333,0,0,0.469120,0,1,0,0,1,0
4,1.000,0.337838,0.2,0.500246,0.000000,1,1,0.395400,0,0,0,1,1,0


In [14]:
X = df1.drop('Exited',axis='columns')
y = df1.Exited

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=10, stratify=y)


In [16]:
X_train.shape

(8000, 13)

In [17]:
X_test.shape

(2000, 13)

In [18]:
import tensorflow as tf
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report

In [19]:
def ANN(X_train, y_train, X_test, y_test, loss):
    model = keras.Sequential([
        keras.layers.Dense(13, input_dim=13, activation='relu'),
        keras.layers.Dense(10, activation='relu'),
        keras.layers.Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    
    model.fit(X_train, y_train, epochs=100)

    print(model.evaluate(X_test, y_test))

    y_preds = model.predict(X_test)
    y_preds = np.round(y_preds)

    print("Classification Report: \n", classification_report(y_test, y_preds))

    return y_preds

In [20]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 0.5732 - accuracy: 0.6900
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4644 - accuracy: 0.7986
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4550 - accuracy: 0.8018
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4460 - accuracy: 0.8066
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4377 - accuracy: 0.8111
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4299 - accuracy: 0.8158
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4242 - accuracy: 0.8146
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4185 - accuracy: 0.8180
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4131 - accuracy: 0.8185
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4043 - accura

250/250 [==============================] - 0s 2ms/step - loss: 0.3367 - accuracy: 0.8605
Epoch 82/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3359 - accuracy: 0.8620
Epoch 83/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3361 - accuracy: 0.8625
Epoch 84/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3348 - accuracy: 0.8640
Epoch 85/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3353 - accuracy: 0.8604
Epoch 86/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3344 - accuracy: 0.8626
Epoch 87/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3353 - accuracy: 0.8608
Epoch 88/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3346 - accuracy: 0.8618
Epoch 89/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3342 - accuracy: 0.8595
Epoch 90/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3347 - accuracy: 

### undersampling
### ================================================

In [21]:
# Class count
count_class_0, count_class_1 = df1.Exited.value_counts()

# Divide by class
df_class_0 = df1[df1['Exited'] == 0]
df_class_1 = df1[df1['Exited'] == 1]

In [22]:
df_class_0_under = df_class_0.sample(count_class_1)
df_test_under = pd.concat([df_class_0_under, df_class_1], axis=0)

print('Random under-sampling:')
print(df_test_under.Exited.value_counts())

Random under-sampling:
0    2037
1    2037
Name: Exited, dtype: int64


In [23]:
from sklearn.model_selection import train_test_split
X = df_test_under.drop('Exited', axis='columns')
y = df_test_under['Exited']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [24]:
y_train.value_counts()

1    1630
0    1629
Name: Exited, dtype: int64

In [25]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
102/102 [==============================] - 1s 2ms/step - loss: 0.6875 - accuracy: 0.5281
Epoch 2/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6595 - accuracy: 0.6352
Epoch 3/100
102/102 [==============================] - 0s 1ms/step - loss: 0.6337 - accuracy: 0.6533
Epoch 4/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6167 - accuracy: 0.6689
Epoch 5/100
102/102 [==============================] - 0s 2ms/step - loss: 0.6061 - accuracy: 0.6778
Epoch 6/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5981 - accuracy: 0.6846
Epoch 7/100
102/102 [==============================] - 0s 2ms/step - loss: 0.5899 - accuracy: 0.6965
Epoch 8/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5837 - accuracy: 0.7033
Epoch 9/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5771 - accuracy: 0.7088
Epoch 10/100
102/102 [==============================] - 0s 1ms/step - loss: 0.5733 - accura

102/102 [==============================] - 0s 1ms/step - loss: 0.4579 - accuracy: 0.7840
Epoch 82/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4584 - accuracy: 0.7769
Epoch 83/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4569 - accuracy: 0.7818
Epoch 84/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4575 - accuracy: 0.7837
Epoch 85/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4562 - accuracy: 0.7834
Epoch 86/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4568 - accuracy: 0.7849
Epoch 87/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4549 - accuracy: 0.7843
Epoch 88/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4569 - accuracy: 0.7818
Epoch 89/100
102/102 [==============================] - 0s 2ms/step - loss: 0.4565 - accuracy: 0.7828
Epoch 90/100
102/102 [==============================] - 0s 1ms/step - loss: 0.4552 - accuracy: 

### Oversampling
### ==============================================


In [26]:
# Oversample 1-class and concat the DataFrames of both classes
df_class_1_over = df_class_1.sample(count_class_0, replace=True)
df_test_over = pd.concat([df_class_0, df_class_1_over], axis=0)

print('Random over-sampling:')
print(df_test_over.Exited.value_counts())

Random over-sampling:
0    7963
1    7963
Name: Exited, dtype: int64


In [27]:
from sklearn.model_selection import train_test_split
X = df_test_over.drop('Exited', axis='columns')
y = df_test_over['Exited']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=15, stratify=y)


In [28]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6554 - accuracy: 0.6189
Epoch 2/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6018 - accuracy: 0.6812
Epoch 3/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5866 - accuracy: 0.6926
Epoch 4/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5799 - accuracy: 0.7018
Epoch 5/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5755 - accuracy: 0.7058
Epoch 6/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5705 - accuracy: 0.7076
Epoch 7/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5639 - accuracy: 0.7133
Epoch 8/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5533 - accuracy: 0.7233
Epoch 9/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5426 - accuracy: 0.7290
Epoch 10/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5319 - accura

399/399 [==============================] - 1s 1ms/step - loss: 0.4424 - accuracy: 0.7917
Epoch 82/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4431 - accuracy: 0.7878
Epoch 83/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4421 - accuracy: 0.7900
Epoch 84/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4416 - accuracy: 0.7908
Epoch 85/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4419 - accuracy: 0.7896
Epoch 86/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4426 - accuracy: 0.7922
Epoch 87/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4418 - accuracy: 0.7927
Epoch 88/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4423 - accuracy: 0.7905
Epoch 89/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4406 - accuracy: 0.7907
Epoch 90/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4401 - accuracy: 

In [29]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [30]:
from imblearn.over_sampling import SMOTE

smote = SMOTE(sampling_strategy='minority')
X_sm, y_sm = smote.fit_resample(X, y)

y_sm.value_counts()

1    7963
0    7963
Name: Exited, dtype: int64

In [31]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_sm, y_sm, test_size=0.2, random_state=15, stratify=y_sm)

In [ ]:
y_preds = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

Epoch 1/100
399/399 [==============================] - 1s 1ms/step - loss: 0.6397 - accuracy: 0.6429
Epoch 2/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5984 - accuracy: 0.6870
Epoch 3/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5756 - accuracy: 0.7035
Epoch 4/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5637 - accuracy: 0.7107
Epoch 5/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5345 - accuracy: 0.7346
Epoch 6/100
399/399 [==============================] - 1s 1ms/step - loss: 0.5042 - accuracy: 0.7533
Epoch 7/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4826 - accuracy: 0.7655
Epoch 8/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4684 - accuracy: 0.7724
Epoch 9/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4608 - accuracy: 0.7768
Epoch 10/100
399/399 [==============================] - 1s 1ms/step - loss: 0.4562 - accura

### Ensemble 
### ====================================================

In [ ]:
df1.Exited.value_counts()

In [ ]:
X = df1.drop('Exited', axis='columns')
y = df1['Exited']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=15, stratify=y)

In [ ]:
df2 = X_train.copy()
df2['Exited'] = y_train

In [ ]:
df2.head()

In [ ]:
df2_class0 = df2[df2.Exited == 0]
df2_class1 = df2[df2.Exited == 1]

In [ ]:
def get_train_batch(df_majority, df_minority, start, end):
    df_train = pd.concat([df_majority[start:end], df_minority], axis=0)

    X_train = df_train.drop('Exited', axis='columns')
    y_train = df_train.Exited
    return X_train, y_train

In [ ]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 0, 1495)

y_pred1 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

In [ ]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 1495, 2990)

y_pred2 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

In [ ]:
X_train, y_train = get_train_batch(df2_class0, df2_class1, 2990, 4130)

y_pred3 = ANN(X_train, y_train, X_test, y_test, 'binary_crossentropy')

In [ ]:
y_pred_final = y_pred1.copy()
for i in range(len(y_pred1)):
    n_ones = y_pred1[i] + y_pred2[i] + y_pred3[i]
    if n_ones > 1:
        y_pred_final[i] = 1
    else:
        y_pred_final[i] = 0

In [ ]:
cl_rep = classification_report(y_test, y_pred_final)
print(cl_rep)